# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [36]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, randint, choice
from sklearn.metrics import mean_squared_error,accuracy_score
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [37]:
subscription_id = 'b4a122b5-b4d5-40e7-9878-57b87adf4a8b'
resource_group = 'aml-quickstarts-125265'
workspace_name = 'quick-starts-ws-125265'

ws = Workspace.get(name="quick-starts-ws-125265")
exp = Experiment(workspace=ws, name="quick-starts-ws-125265")

In [38]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    comp_cluster = ComputeTarget(workspace=ws, name='Ranga')
    print('Cluster already exists')
except ComputeTargetException:
    comp_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)
    comp_cluster = ComputeTarget.create(ws,'Ranga',comp_config)

comp_cluster.wait_for_completion(show_output=True)

Cluster already exists
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for choosing the different hyperparameters, termination policy and config settings.


In [39]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.01)

another_early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    'max_depth': choice(3,9),
    'learning_rate': choice(0.01,10)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(entry_script='train_FINAL.py',source_directory='./training',compute_target=comp_cluster)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,primary_metric_name='R-square',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,policy=early_termination_policy,
estimator=estimator,max_concurrent_runs=4)

In [40]:
#TODO: Submit your experiment
run = exp.submit(hyperdrive_run_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [42]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cb729ede-a7d9-4db1-9468-c754c6736649
Web View: https://ml.azure.com/experiments/quick-starts-ws-125265/runs/HD_cb729ede-a7d9-4db1-9468-c754c6736649?wsid=/subscriptions/b4a122b5-b4d5-40e7-9878-57b87adf4a8b/resourcegroups/aml-quickstarts-125265/workspaces/quick-starts-ws-125265

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-08T19:23:17.536753][API][INFO]Experiment created<END>\n""<START>[2020-11-08T19:23:18.230464][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-08T19:23:18.545738][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-08T19:23:18.5578453Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END><START>[2020-11-08T19:23:49.0015145Z][SCHEDULER][INFO]The execution environment was successfully prepared.<END><START>[2020-11-08T19:23:49.0976397Z][SCHEDULER][INFO]Scheduling job, id='HD_c

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [16]:
best = run.get_best_run_by_primary_metric() #Best Run

In [ ]:
#TODO: Save the best model
joblib.dump(best,'best_hyperparam.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service